# Bubbles example

In [ ]:
import mefikit as mf
import numpy as np
import pyvista as pv

rng = np.random.default_rng(seed=123)
pv.set_plot_theme("dark")
pv.set_jupyter_backend("static")

In [ ]:
xmax = 5.0
ymax = 1.0
r = 0.17
nb = 15
nr = 12.5

In [ ]:
nx = int(xmax / r * nr)
ny = int(ymax / r * nr)
print(f"Number of elements : {nx * ny * ny:,}")

In [ ]:
xc = rng.uniform(r, xmax - r, nb)
yc = rng.uniform(r, ymax - r, nb)
zc = rng.uniform(r, ymax - r, nb)
spheres = [mf.sel.sphere([x, y, z], r) for x, y, z in zip(xc, yc, zc)]
sphere_union = spheres[0]
for s in spheres[1:]:
    sphere_union = sphere_union | s

In [ ]:
x = np.linspace(0.0, xmax, nx)
y = np.linspace(0.0, ymax, ny)
volumes = mf.build_cmesh(x, y, y)

In [ ]:
inner_bubbles = volumes.select(sphere_union)
interface = inner_bubbles.boundaries()
cracked = volumes.crack(interface)

In [ ]:
cracked.boundaries().to_pyvista().plot(opacity=0.4)

In [ ]:
compos = cracked.connected_components()

In [ ]:
pv.set_plot_theme("default")
pv.set_jupyter_backend("static")
pv.global_theme.color_cycler = "default"
pl = pv.Plotter()
for i, c in enumerate(compos[1:]):
    compo = c.to_pyvista()
    pl.add_mesh(compo, opacity=0.4)
    # pl.add_point_labels(compo.center, [f"Mesh n°{i}"])
# pl.camera_position = "xz"
# pl.camera.position = (xmax/2.0, -xmax/0.6, ymax/2.0)

pl.show()
pv.global_theme.color_cycler = None